In [1]:
# FIX yahoo Finance Error
!pip install curl_cffi
from curl_cffi import requests
session = requests.Session(impersonate="chrome")

In [2]:
# Import necessary libraries
import os
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import time
from tenacity import retry, stop_after_attempt, wait_fixed, retry_if_exception_type

In [3]:
# Section 0: ILLIQ is constructed as equal-weighted High-Low Amihud portfolios to capture small-cap liquidity effects.

In [4]:
def compute_t_stat(series, annualization_factor=252):
    mean = series.mean() * annualization_factor * 100  # Annualized premium in percentage
    std = series.std() * np.sqrt(annualization_factor) * 100
    n = len(series)
    t_stat = mean / (std / np.sqrt(n))
    p_value = 2 * (1 - stats.t.cdf(np.abs(t_stat), n - 1))
    return mean, t_stat, p_value

# Load and compute ILLIQ statistics
illiq_factor_vw = pd.read_csv(os.path.join(notebook_dir, "illiq_factor_vw.csv"))
illiq_factor_vw['date'] = pd.to_datetime(illiq_factor_vw['date'])
illiq_factor_vw = illiq_factor_vw.set_index('date')['ILLIQ_VW']
mean_vw, t_stat_vw, p_value_vw = compute_t_stat(illiq_factor_vw)

illiq_factor_ew = pd.read_csv(os.path.join(notebook_dir, "illiq_factor_ew.csv"))
illiq_factor_ew['date'] = pd.to_datetime(illiq_factor_ew['date'])
illiq_factor_ew = illiq_factor_ew.set_index('date')['ILLIQ_EW']
mean_ew, t_stat_ew, p_value_ew = compute_t_stat(illiq_factor_ew)

print("\nIlliquidity Premium Statistics (2017–2024):")
print(f"Value-Weighted ILLIQ: Premium = {mean_vw:.4f}%, t-stat = {t_stat_vw:.4f}, p-value = {p_value_vw:.4f}")
print(f"Equal-Weighted ILLIQ: Premium = {mean_ew:.4f}%, t-stat = {t_stat_ew:.4f}, p-value = {p_value_ew:.4f}")


Illiquidity Premium Statistics (2017–2024):
Value-Weighted ILLIQ: Premium = -9.6663%, t-stat = -23.2507, p-value = 0.0000
Equal-Weighted ILLIQ: Premium = 0.6098%, t-stat = 2.9605, p-value = 0.0031


In [5]:
# Load Fama-French data
ff_data = pd.read_csv(r"fama_french_data.csv")
ff_data['date'] = pd.to_datetime(ff_data['date'])
ff_data = ff_data.set_index('date')

# Compute simple average premiums for all factors
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom', 'ILLIQ']
simple_avg_premiums = ff_data[factors].mean() * 252 * 100

print("\nSimple Average Premiums for All Factors (2017–2024):")
for factor, premium in simple_avg_premiums.items():
    print(f"{factor}: {premium:.4f}%")


Simple Average Premiums for All Factors (2017–2024):
Mkt-RF: 12.3985%
SMB: -2.9770%
HML: -3.2496%
RMW: 4.4295%
CMA: -0.4460%
Mom: -0.0000%
ILLIQ: 0.0000%


In [6]:
# --- Section 1: Data Preparation ---
# This section loads and cleans data, computes factor premiums, and saves results for later use.

In [7]:
# 1.1: CRSP Data Cleaning
def clean_crsp_data(df):
    df = df.copy()
    
    # Convert returns to numeric
    df["RET"] = pd.to_numeric(df["RET"], errors="coerce")
    df["DLRET"] = pd.to_numeric(df["DLRET"], errors="coerce")
    
    # Drop rows with missing critical values
    df = df.dropna(subset=["PRC", "VOL", "SHROUT", "CFACPR", "CFACSHR"])
    
    # Handle delisting returns
    df["DLSTCD"] = df["DLSTCD"].fillna(0)
    mask = df["DLSTCD"].between(200, 399)
    df["RET"] = np.where(mask, df["DLRET"], df["RET"])
    df["RET"] = df["RET"].fillna(-0.30)
    
    # Adjust for corporate actions
    df["CFACPR"] = df["CFACPR"].replace(0, 1)
    df["CFACSHR"] = df["CFACSHR"].replace(0, 1)
    df["adj_prc"] = df["PRC"] / df["CFACPR"]
    df["adj_shrout"] = (df["SHROUT"] * 1000) * df["CFACSHR"]
    df["adj_volume"] = df["VOL"] * df["CFACSHR"]
    
    # Convert to float32
    df["adj_prc"] = df["adj_prc"].astype("float32")
    df["adj_volume"] = df["adj_volume"].astype("float32")
    
    # Calculate dollar volume
    df["dollar_volume"] = df["adj_prc"].abs() * df["adj_volume"]
    
    # Handle invalid values
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Calculate daily market cap
    df["daily_market_cap"] = df["adj_prc"] * df["adj_shrout"]
    df["daily_market_cap"] = df["daily_market_cap"].where(df["daily_market_cap"] >= 0, np.nan)
    
    # Apply filters to remove if exceptional return or volume does not makes sense
    df["RET"] = np.where(df["RET"].abs() > 2, np.nan, df["RET"])
    df["adj_volume"] = np.where(df["adj_volume"] > df["adj_shrout"], np.nan, df["adj_volume"])
    df["dollar_volume"] = df["adj_prc"].abs() * df["adj_volume"]
    df["adj_volume"] = np.where(df["dollar_volume"] < 100, np.nan, df["adj_volume"])
    
    # Customized filters to reduce more noices
    # 1. Exclude microcaps (market cap < $50M)
    df = df[df["daily_market_cap"] >= 50_000_000]
    # 2. Exclude stocks with low trading volume (dollar volume < $1M)
    df = df[df["dollar_volume"] >= 1_000_000]
    # 3. Winsorize returns to reduce impact of outliers
    df["RET"] = df["RET"].clip(lower=df["RET"].quantile(0.01), upper=df["RET"].quantile(0.99))
    
    df = df.dropna(subset=["RET", "adj_volume", "daily_market_cap"])
    
    return df

In [8]:
# 1.2: Load Fama-French Data and ILLIQ Factor
def load_fama_french_data(ff_path, mom_path, illiq_path):
    # Load Fama-French 5-factor data
    ff_data = pd.read_csv(ff_path)
    
    # Strip whitespace from column names
    ff_data.columns = ff_data.columns.str.strip()
    
    # Verify expected columns for 5-factor data
    ff_expected_columns = ['date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
    if not all(col in ff_data.columns for col in ff_expected_columns):
        raise ValueError(f"Fama-French 5-Factor CSV missing expected columns. Found: {ff_data.columns}, Expected: {ff_expected_columns}")
    
    # Load Momentum factor data
    mom_data = pd.read_csv(mom_path)
    
    # Strip whitespace from column names
    mom_data.columns = mom_data.columns.str.strip()
    
    # Verify expected columns for Momentum data
    mom_expected_columns = ['date', 'Mom']
    if not all(col in mom_data.columns for col in mom_expected_columns):
        raise ValueError(f"Momentum CSV missing expected columns. Found: {mom_data.columns}, Expected: {mom_expected_columns}")
    
    # Load ILLIQ factor data (value-weighted)
    illiq_data = pd.read_csv(illiq_path)
    
    # Strip whitespace from column names
    illiq_data.columns = illiq_data.columns.str.strip()
    
    # Verify expected columns for ILLIQ data
    illiq_expected_columns = ['date', 'ILLIQ_VW']
    if not all(col in illiq_data.columns for col in illiq_expected_columns):
        raise ValueError(f"ILLIQ CSV missing expected columns. Found: {illiq_data.columns}, Expected: {illiq_expected_columns}")
    
    # Rename ILLIQ_VW to ILLIQ for consistency in the app
    illiq_data = illiq_data.rename(columns={'ILLIQ_VW': 'ILLIQ'})
    
    # Convert dates to datetime
    ff_data['date'] = pd.to_datetime(ff_data['date'], format='%Y%m%d')
    mom_data['date'] = pd.to_datetime(mom_data['date'], format='%Y%m%d')
    illiq_data['date'] = pd.to_datetime(illiq_data['date'])
    
    # Merge the datasets on date
    ff_data = ff_data.merge(mom_data[['date', 'Mom']], on='date', how='inner')
    ff_data = ff_data.merge(illiq_data[['date', 'ILLIQ']], on='date', how='inner')
    ff_data = ff_data.set_index('date')[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom', 'ILLIQ']]
    
    # Convert to decimals (assuming Fama-French and Momentum data are in percentage points)
    ff_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom', 'RF']] = ff_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom', 'RF']] / 100
    
    # Winsorize factor returns to reduce outliers (1st and 99th percentiles)
    factors_to_winsorize = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom', 'ILLIQ']
    for factor in factors_to_winsorize:
        ff_data[factor] = ff_data[factor].clip(lower=ff_data[factor].quantile(0.01), upper=ff_data[factor].quantile(0.99))

    # Orthogonalize ILLIQ and Mom against correlated factors    
    # Orthogonalize ILLIQ against Mkt-RF, HML, and CMA (correlations > 0.2)
    X = sm.add_constant(ff_data[['Mkt-RF', 'HML', 'CMA']])
    model = sm.OLS(ff_data['ILLIQ'], X).fit()
    ff_data['ILLIQ'] = model.resid
    
    # Orthogonalize Mom against SMB and HML (correlations > 0.3)
    X = sm.add_constant(ff_data[['SMB', 'HML']])
    model = sm.OLS(ff_data['Mom'], X).fit()
    ff_data['Mom'] = model.resid
    
    # Check for missing values in factor returns
    if ff_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom', 'ILLIQ', 'RF']].isna().any().any():
        raise ValueError("Fama-French data contains missing values in factor returns or RF. Please ensure the data is complete.")
    
    return ff_data

In [9]:
# 1.3: Fama-MacBeth for All Factors
def run_fama_macbeth(crsp_data, ff_data, significance_threshold=0.05):
    """
    Run Fama-MacBeth regression for all factors using the entire US stock universe.
    
    Notes:
    - Factors include Mkt-RF, SMB, HML, RMW, CMA, Mom, and ILLIQ (constructed as High Amihud minus Low Amihud portfolio returns, value-weighted).
    - Stock excess returns (R_i - R_f) are sourced from CRSP data.
    """
    # Clean CRSP data
    crsp_data = clean_crsp_data(crsp_data)
    
    # Check sample size
    unique_stocks = crsp_data['PERMNO'].nunique()
    if unique_stocks < 50:
        raise ValueError(f"Insufficient stocks in CRSP data ({unique_stocks} < 50). Please ensure the CRSP dataset contains enough stocks.")
    
    # Align dates
    common_dates = crsp_data.index.intersection(ff_data.index)
    if len(common_dates) < 252:
        raise ValueError(f"Insufficient overlapping dates between CRSP and Fama-French data ({len(common_dates)} < 252 trading days).")
    
    crsp_data = crsp_data.loc[common_dates]
    ff_data = ff_data.loc[common_dates]
    
    # Define all factors (using value-weighted ILLIQ)
    factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom', 'ILLIQ']
    
    # Check for multicollinearity by computing factor correlations
    print("Factor Correlations (2017–2024):")
    print(ff_data[factors].corr())
    
    # Step 1: Time-series regressions to estimate betas for each stock
    betas_dict = {}
    for permno in crsp_data['PERMNO'].unique():
        stock_data = crsp_data[crsp_data['PERMNO'] == permno]
        if len(stock_data) < 100:
            continue
        y = stock_data['RET'] - ff_data['RF']
        X_factors = ff_data[factors]
        
        data = pd.concat([y, X_factors], axis=1).replace([np.inf, -np.inf], np.nan).dropna()
        if data.empty:
            continue
        
        y = data.iloc[:, 0]
        X = sm.add_constant(data.iloc[:, 1:])
        model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 1})
        betas_dict[permno] = model.params.drop('const')
    
    beta_df = pd.DataFrame(betas_dict).T
    if beta_df.empty:
        raise ValueError("No valid betas estimated. Check CRSP data for sufficient stock observations.")
    
    # Step 2: Cross-sectional regressions to estimate factor premiums
    premiums = []
    for date in common_dates:
        daily_data = crsp_data[crsp_data.index == date]
        if len(daily_data) < 10:
            continue
        y = daily_data['RET'] - ff_data.loc[date, 'RF']
        valid_permnos = daily_data['PERMNO'].isin(beta_df.index)
        if not valid_permnos.any():
            continue
        
        # Get the subset of data for valid PERMNOs
        daily_data_valid = daily_data[valid_permnos]
        y_valid = y[valid_permnos]
        
        # Reindex y_valid by PERMNO to match X's index
        y_valid = pd.Series(y_valid.values, index=daily_data_valid['PERMNO'])
        
        # Ensure the index (PERMNO) is unique
        if y_valid.index.duplicated().any():
            raise ValueError(f"Duplicate PERMNOs found on date {date}. Please ensure CRSP data has unique PERMNOs per date.")
        
        X = sm.add_constant(beta_df.loc[daily_data_valid['PERMNO'], factors])
        
        # Concatenate y_valid and X, which are now both indexed by PERMNO
        data = pd.concat([y_valid, X], axis=1).replace([np.inf, -np.inf], np.nan).dropna()
        if data.empty:
            continue
        
        y = data.iloc[:, 0]
        X = data.iloc[:, 1:]
        model = sm.OLS(y, X).fit()
        premiums.append(model.params.drop('const'))
    
    premium_df = pd.DataFrame(premiums, columns=factors)
    if premium_df.empty:
        raise ValueError("No valid premiums estimated. Ensure there are enough cross-sectional observations.")
    
    # Compute statistics and apply significance test
    final_premiums = {}
    significant_factors = []
    print("\nFama-MacBeth Results (2017–2024):")
    for factor in factors:
        avg_premium = premium_df[factor].mean() * 252 * 100  # Annualized premium in percentage
        std_premium = premium_df[factor].std() * np.sqrt(252) * 100
        n = len(premium_df)
        t_stat = avg_premium / (std_premium / np.sqrt(n))
        p_value = 2 * (1 - stats.t.cdf(np.abs(t_stat), n - 1))
        
        # Display premium with higher precision
        print(f"{factor}: Premium = {avg_premium:.8f}%, t-stat = {t_stat:.4f}, p-value = {p_value:.4f}")
        if p_value < significance_threshold:
            final_premiums[factor] = avg_premium
            significant_factors.append(factor)
        else:
            print(f"{factor} excluded (p = {p_value:.4f} > {significance_threshold})")
    
    return pd.Series(final_premiums), significant_factors

In [10]:
# 1.4: Data Preparation Main Function
def prepare_data(ff_path, mom_path, illiq_path, crsp_path):
    """Prepare data by loading, cleaning, and computing factor premiums."""
    # Load CRSP data
    dtypes = {
        "PERMNO": "Int64",
        "RET": "object",
        "DLRET": "object",
        "PRC": "float64",
        "VOL": "float64",
        "SHROUT": "float64",
        "SHRCD": "Int16",
        "EXCHCD": "Int8",
        "CFACPR": "float64",
        "CFACSHR": "float64",
        "DLSTCD": "Int16"
    }
    crsp_data = pd.read_csv(
        crsp_path,
        dtype=dtypes
    )
    crsp_data['date'] = pd.to_datetime(crsp_data['date'], format='%Y%m%d')
    crsp_data = crsp_data.set_index('date')
    
    # Load Fama-French, Momentum, and ILLIQ data
    ff_data = load_fama_french_data(ff_path, mom_path, illiq_path)
    
    # Run Fama-MacBeth for all factors
    all_premiums, significant_factors = run_fama_macbeth(crsp_data, ff_data)
    
    # Save results
    all_premiums.to_csv(os.path.join(notebook_dir, "factor_premiums.csv"), float_format='%.8f')
    pd.Series(significant_factors).to_csv(os.path.join(notebook_dir, "significant_factors.csv"), index=False)
    ff_data.to_csv(os.path.join(notebook_dir, "fama_french_data.csv"))
    
    print("\nData preparation complete. Files saved:")
    print(f"- {os.path.join(notebook_dir, 'factor_premiums.csv')}")
    print(f"- {os.path.join(notebook_dir, 'significant_factors.csv')}")
    print(f"- {os.path.join(notebook_dir, 'fama_french_data.csv')}")
    
    return all_premiums, significant_factors, ff_data

In [11]:
# --- Section 2: Stock Analysis ---
# This section analyzes a specific stock using the pre-computed premiums.

In [12]:
# 2: Get Market VIX for fear estimation
@retry(
    stop=stop_after_attempt(3),
    wait=wait_fixed(5),
    retry=retry_if_exception_type(Exception),
    before_sleep=lambda retry_state: print(f"Retrying VIX fetch (attempt {retry_state.attempt_number}/3)...")
)
def get_current_vix():
    """Fetch the current VIX value from Yahoo Finance with retries.
    
    Returns:
        float: Current VIX value, or 20.0 if fetch fails after retries.
    """
    try:
        time.sleep(1)  # Delay to avoid rate limits
        vix = yf.Ticker("^VIX", session = session)
        vix_data = vix.history(period="1d")
        if vix_data.empty:
            raise ValueError("No VIX data available.")
        current_vix = vix_data['Close'].iloc[-1]
        return current_vix
    except Exception as e:
        print(f"Error fetching VIX data: {str(e)}")
        if retry_state.attempt_number == 3:
            print("All VIX fetch attempts failed. Using default VIX value: 20.0")
            return 20.0
        raise  # Retry if attempts remain

def get_market_regime(vix, low_threshold=15, high_threshold=25):
    """Determine the market regime based on VIX level.
    
    Args:
        vix (float): Current VIX value.
        low_threshold (float): Threshold for low volatility (default: 15).
        high_threshold (float): Threshold for high volatility (default: 25).
    
    Returns:
        str: 'low', 'medium', or 'high' volatility.
    """
    if vix is None:
        return 'medium'  # Default to medium if VIX fetch fails
    if vix < low_threshold:
        return 'low'
    elif vix < high_threshold:
        return 'medium'
    else:
        return 'high'

In [13]:
# 2.1: Get Stock Data
@retry(
    stop=stop_after_attempt(3),
    wait=wait_fixed(5),
    retry=retry_if_exception_type(Exception),
    before_sleep=lambda retry_state: print(f"Retrying {ticker} fetch (attempt {retry_state.attempt_number}/3)...")
)
def fetch_stock_data(ticker, start_date, end_date):
    """Fetch historical stock data from Yahoo Finance with retries and error handling.
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL').
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
    
    Returns:
        pd.DataFrame: DataFrame with columns 'Close', 'RET', 'volume_dollar', indexed by 'date'.
                     Returns empty DataFrame if fetch fails after retries.
    """
    try:
        time.sleep(1)  # Delay to avoid rate limits
        stock = yf.Ticker(ticker, session = session)
        df = stock.history(start=start_date, end=end_date, raise_errors=True)
        
        if df.empty:
            raise ValueError(f"No data available for {ticker} between {start_date} and {end_date}.")
        
        df.reset_index(inplace=True)
        df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None)
        
        df = df.rename(columns={
            'Date': 'date',
            'Close': 'Close',
            'Volume': 'Volume'
        })
        
        df = df.set_index('date')
        
        df['RET'] = df['Close'].pct_change()
        df['volume_dollar'] = df['Volume'] * df['Close']
        
        df = df[['Close', 'RET', 'volume_dollar']].dropna()
        
        return df
    
    except Exception as e:
        print(f"Error fetching data for {ticker}: {str(e)}")
        if retry_state.attempt_number == 3:
            print(f"All fetch attempts failed for {ticker}. Returning empty DataFrame.")
            return pd.DataFrame(columns=['Close', 'RET', 'volume_dollar'])
        raise

In [14]:
# 2.2: Calculate Amihud Illiquidity (for reporting purposes only, not used as a factor)
def calculate_amihud_illiquidity(df):
    """Calculate the Amihud illiquidity ratio for a stock (scaled by 1e6 for consistency)."""
    df['abs_return'] = df['RET'].abs()
    df['amihud'] = np.where(
        df['volume_dollar'] == 0,
        np.nan,
        (df['abs_return'] / df['volume_dollar']) * 1e6
    )
    # Keep 'RET' in the output DataFrame
    return df[['amihud', 'RET']].replace([np.inf, -np.inf], np.nan).dropna()

In [15]:
# 2.3: Calculate Stock Betas
def calculate_stock_betas(ticker, stock_data, ff_data, factors):
    """Calculate factor betas for a single stock, returning betas and p-values.
    
    Args:
        ticker (str): Stock ticker symbol.
        stock_data (pd.DataFrame): Stock data with columns 'RET', 'amihud', 'Ticker'.
        ff_data (pd.DataFrame): Factor data with columns 'RF', factor columns, indexed by date.
        factors (list of str): List of factor names (e.g., ['Mkt-RF', 'SMB', ...]).
    
    Returns:
        tuple: (pd.Series of betas, pd.Series of p-values).
    """
    # Align dates using merge
    aligned_data = pd.merge(
        stock_data.reset_index(),
        ff_data.reset_index(),
        on='date',
        how='inner'
    ).set_index('date')
    
    if len(aligned_data) < 252:
        print(f"Warning: Limited data for {ticker} ({len(aligned_data)} days).")
    
    # Check if required columns exist
    required_columns = ['RET', 'RF'] + factors
    missing_columns = [col for col in required_columns if col not in aligned_data.columns]
    if missing_columns:
        print(f"Error: Missing columns {missing_columns} for {ticker}. Using default betas.")
        default_betas = pd.Series(1.0 if f == 'Mkt-RF' else 0.0, index=['const'] + factors)
        default_pvalues = pd.Series(1.0, index=['const'] + factors)
        return default_betas, default_pvalues
    
    y = aligned_data['RET'] - aligned_data['RF']
    X_factors = aligned_data[factors]
    X = sm.add_constant(X_factors)
    
    data = pd.concat([y, X], axis=1).replace([np.inf, -np.inf], np.nan).dropna()
    if data.empty:
        print(f"Warning: No valid data for {ticker}. Using default betas.")
        default_betas = pd.Series(1.0 if f == 'Mkt-RF' else 0.0, index=['const'] + factors)
        default_pvalues = pd.Series(1.0, index=['const'] + factors)
        return default_betas, default_pvalues
    
    y = data.iloc[:, 0]
    X = data.iloc[:, 1:]
    model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 1})
    
    betas = model.params
    pvalues = model.pvalues
    for factor in betas.index:
        if pvalues[factor] > 0.05:
            print(f"Note: {factor} beta for {ticker} is not significant (p = {pvalues[factor]:.4f}).")
    
    return betas, pvalues

In [16]:
# 2.4: Calculate Expected Return
def calculate_expected_return(stock_data, ff_data, avg_premiums, factors):
    """Calculate expected return using the multi-factor model, excluding Mom factor.
    
    Args:
        stock_data (pd.DataFrame): Stock data with columns 'RET', 'amihud', 'Ticker'.
        ff_data (pd.DataFrame): Factor data with columns 'RF', factor columns, indexed by date.
        avg_premiums (pd.Series): Fama-MacBeth factor premiums (in %).
        factors (list of str): List of factor names.
    
    Returns:
        tuple: (float: expected return, dict: factor contributions, pd.Series: betas, pd.Series: p-values).
    """
    # Align dates
    aligned_data = pd.merge(
        stock_data.reset_index(),
        ff_data.reset_index(),
        on='date',
        how='inner'
    ).set_index('date')
    
    # Check if stock_data is empty or missing Ticker
    if stock_data.empty or 'Ticker' not in stock_data.columns:
        print("Error: Stock data is empty or missing Ticker column. Cannot compute expected return.")
        return 0.0, {}, pd.Series(), pd.Series()
    
    ticker = stock_data['Ticker'].iloc[0]
    betas, pvalues = calculate_stock_betas(ticker, stock_data, ff_data, factors)
    
    rf = aligned_data['RF'].mean() * 252
    
    expected_return = rf
    contributions = {}
    for factor in factors:
        premium = avg_premiums.get(factor, 0.0) / 100
        # Cap the Mom premium at 10% (a more reasonable historical value)
        if factor == 'Mom':
            premium = min(premium, 0.10)  # 10% annual premium
            print(f"Note: Capped Mom premium at 10% for {ticker} (original: {avg_premiums[factor]}%).")
        beta = betas.get(factor, 0.0)
        contribution = beta * premium
        expected_return += contribution
        contributions[factor] = contribution * 100
    
    return expected_return, contributions, betas, pvalues

In [17]:
# 2.5: Analyze a Specific Stock
def advise_portfolio(ticker, start_date, end_date):
    """Provide portfolio advice for a specific stock using the multi-factor model.

    Args:
        ticker (str): Stock ticker symbol.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
    
    Returns:
        dict: Analysis results including expected return, betas, p-values, and illiquidity.
    
    Note on factor premiums:
    - Factor premiums (Mkt-RF, SMB, HML, RMW, CMA, Mom, ILLIQ) are estimated using Fama-MacBeth
      on the entire US stock universe (CRSP data).
    - Factor returns for Mkt-RF, SMB, HML, RMW, CMA, and Mom are sourced from Fama-French CSVs.
    - ILLIQ is constructed as High Amihud minus Low Amihud portfolio returns (value-weighted, including NASDAQ).
    - Stock-specific variation in risk exposure is captured by the betas, estimated for each stock.
    - ILLIQ and Mom are orthogonalized: ILLIQ against Mkt-RF, HML, CMA; Mom against SMB, HML, CMA.
      Their contributions reflect exposure to these orthogonalized components.
    """
    try:
        # Load pre-computed data
        avg_premiums = pd.read_csv("factor_premiums.csv", index_col=0).squeeze()
        significant_factors = pd.read_csv("significant_factors.csv").squeeze().tolist()
        ff_data = pd.read_csv("fama_french_data.csv").set_index('date')
        ff_data.index = pd.to_datetime(ff_data.index)
        
        # Verify date range overlap
        ff_start = ff_data.index.min()
        ff_end = ff_data.index.max()
        start = pd.to_datetime(start_date)
        end = pd.to_datetime(end_date)
        if start > ff_end or end < ff_start:
            raise ValueError(f"No overlapping dates between stock data ({start_date} to {end_date}) and factor data ({ff_start} to {ff_end}).")
        
        # Fetch stock data
        df = fetch_stock_data(ticker, start_date, end_date)
        
        # Calculate Amihud (for reporting purposes only)
        amihud_data = calculate_amihud_illiquidity(df)
        amihud_data['Ticker'] = ticker
        
        # Calculate expected return
        expected_return, contributions, betas, pvalues = calculate_expected_return(
            amihud_data, ff_data, avg_premiums, significant_factors
        )
        
        # Output advice
        advice = {
            'Ticker': ticker,
            'Period': f"{start_date} to {end_date}",
            'Amihud Illiquidity': amihud_data['amihud'].mean(),
            'Betas': betas.to_dict(),
            'Beta P-Values': pvalues.to_dict(),
            'Expected Annual Return (%)': expected_return * 100,
            'Factor Contributions (%)': contributions
        }
        
        return advice
    
    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None

In [18]:
# 2.6: Compute Sharpe Ratio of Portfolio
def compute_historical_sharpe(tickers, weights, start_date, end_date, rf=0.02):
    """Compute historical Sharpe ratios for a portfolio and the S&P 500 over a specified period.
    
    Parameters:
        tickers (list of str): List of stock ticker symbols.
        weights (list of float): List of weights corresponding to each stock.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        rf (float): Annual risk-free rate, default is 0.02 (2%).
    
    Returns:
        tuple:
            list of float: Portfolio Sharpe ratios for each month.
            list of float: S&P 500 Sharpe ratios for each month.
            list of str: Labels for each month (e.g., 'Jan', 'Feb').
    
    Side Effects:
        Prints messages for data fetch failures or default value usage.
    """
    portfolio_rets = pd.DataFrame()
    successful_tickers = []
    successful_weights = []
    
    for ticker, weight in zip(tickers, weights):
        try:
            rets = fetch_stock_data(ticker, start_date, end_date)  # Use fetch_stock_data
            if not rets.empty:
                portfolio_rets[ticker] = rets['RET']  # Use RET column
                successful_tickers.append(ticker)
                successful_weights.append(weight)
        except Exception as e:
            print(f"Skipping {ticker} for historical Sharpe calculation: {str(e)}")
            continue
    
    sp500_rets = fetch_stock_data('^GSPC', start_date, end_date)  # Use fetch_stock_data
    if sp500_rets.empty:
        print("Warning: No S&P 500 data. Using default Sharpe values.")
        return [0.5, 0.6, 0.8, 0.7], [0.4, 0.5, 0.5, 0.6], ['Jan', 'Feb', 'Mar', 'Apr']
    
    total_weight = sum(successful_weights)
    if total_weight == 0:
        successful_weights = [1.0 / len(successful_tickers)] * len(successful_tickers)
    else:
        successful_weights = [w / total_weight for w in successful_weights]
    
    portfolio_rets['Portfolio'] = portfolio_rets.dot(successful_weights)
    monthly_periods = pd.date_range(start=start_date, end=end_date, freq='ME')
    portfolio_sharpes = []
    sp500_sharpes = []
    labels = []
    
    for i in range(len(monthly_periods) - 1):
        start = monthly_periods[i]
        end = monthly_periods[i + 1]
        period_portfolio = portfolio_rets['Portfolio'].loc[start:end]
        period_sp500 = sp500_rets['RET'].loc[start:end]  # Use RET column
        if not period_portfolio.empty and not period_sp500.empty:
            portfolio_ret = period_portfolio.mean() * 252
            portfolio_vol = period_portfolio.std() * np.sqrt(252)
            sp500_ret = period_sp500.mean() * 252
            sp500_vol = period_sp500.std() * np.sqrt(252)
            portfolio_sharpe = (portfolio_ret - rf) / portfolio_vol if portfolio_vol != 0 else 0
            sp500_sharpe = (sp500_ret - rf) / sp500_vol if sp500_vol != 0 else 0
            portfolio_sharpes.append(portfolio_sharpe)
            sp500_sharpes.append(sp500_sharpe)
            labels.append(start.strftime('%b'))
    
    if not portfolio_sharpes:
        print("No overlapping data for Sharpe ratio calculation. Using default values.")
        return [0.5, 0.6, 0.8, 0.7], [0.4, 0.5, 0.5, 0.6], ['Jan', 'Feb', 'Mar', 'Apr']
    
    return portfolio_sharpes, sp500_sharpes, labels
def plot_sharpe_tracker(portfolio_sharpes, sp500_sharpes, labels):
    """Create a line chart tracking the portfolio's Sharpe ratio vs. the S&P 500 over time.
    
    Parameters:
        portfolio_sharpes (list of float): Portfolio Sharpe ratios.
        sp500_sharpes (list of float): S&P 500 Sharpe ratios.
        labels (list of str): Time labels for the x-axis (e.g., ['Jan', 'Feb']).
    
    Returns:
        None
    
    Side Effects:
        Saves a chart as 'sharpe_tracker.png' in the notebook directory and prints a confirmation message.
    """
    plt.figure(figsize=(8, 6))
    plt.plot(labels, portfolio_sharpes, label='Portfolio Sharpe Ratio', color='#4CAF50')
    plt.plot(labels, sp500_sharpes, label='S&P 500 Sharpe Ratio', color='#2196F3')
    plt.title('Sharpe Ratio Tracker')
    plt.ylabel('Sharpe Ratio')
    plt.xlabel('Month')
    plt.legend()
    plt.ylim(0, max(max(portfolio_sharpes, default=0), max(sp500_sharpes, default=0)) * 1.2)
    output_path = os.path.join(notebook_dir, 'sharpe_tracker.png')
    plt.savefig(output_path)
    plt.close()
    print(f"Sharpe ratio tracker chart saved as {output_path}.")

In [19]:
# 2.7: Weight Allocation for Portfolio and Output Feature
def suggest_portfolio(tickers, start_date, end_date, risk_tolerance='moderate', min_weight=0.1, max_weight=0.5):
    """Suggest portfolio holdings by analyzing stocks and computing allocations based on risk-adjusted scores.
    
    Parameters:
        tickers (list of str): List of stock ticker symbols (e.g., ['META', 'AAPL']).
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        risk_tolerance (str): User's risk tolerance, one of 'conservative', 'moderate', 'aggressive', default is 'moderate'.
        min_weight (float): Minimum allocation weight per stock, default is 0.1.
        max_weight (float): Maximum allocation weight per stock, default is 0.5.
    
    Returns:
        dict: Portfolio summary with keys:
              'Allocations' (dict of str to float), 'Portfolio Expected Return (%)' (float),
              'Portfolio Factor Exposures' (dict of str to float), 'Portfolio Amihud Illiquidity' (float),
              'Individual Stock Analysis' (dict of str to dict), 'Market Regime' (str),
              'Risk Tolerance' (str), 'Insights' (list of str), 'Historical Sharpe' (dict),
              'Weights' (list of float). Returns None if processing fails.
    
    Side Effects:
        Prints VIX status, errors, and warnings during processing.
    """
    try:
        avg_premiums = pd.read_csv("factor_premiums.csv", index_col=0).squeeze()
        significant_factors = pd.read_csv("significant_factors.csv").squeeze().tolist()
        ff_data = pd.read_csv("fama_french_data.csv").set_index('date')
        ff_data.index = pd.to_datetime(ff_data.index).tz_localize(None)
        
        current_vix = get_current_vix()
        if current_vix is None:
            print("Warning: Could not fetch VIX data. Assuming medium volatility.")
            market_regime = 'medium'
        else:
            market_regime = get_market_regime(current_vix)
            print(f"Current VIX: {current_vix:.2f}, Market Regime: {market_regime}")
        
        stock_results = []
        for ticker in tickers:
            result = advise_portfolio(ticker, start_date, end_date)
            if result:
                stock_results.append(result)
            else:
                print(f"Skipping {ticker} due to errors in analysis.")
        
        if not stock_results:
            raise ValueError("No valid stock analysis results to build portfolio.")
        
        rf = ff_data['RF'].mean() * 252
        scores = []
        for result in stock_results:
            ticker = result['Ticker']
            expected_return = result['Expected Annual Return (%)'] / 100
            mkt_beta = result['Betas'].get('Mkt-RF', 1.0)
            amihud = result['Amihud Illiquidity']
            sharpe = (expected_return - rf) / mkt_beta if mkt_beta != 0 else 0
            illiquidity_penalty = 1 / (1 + amihud * 1e6)
            risk_adjustment = (0.5 if market_regime == 'high' else 1.0) if risk_tolerance == 'conservative' else \
                              (1.5 if market_regime == 'low' else 1.0) if risk_tolerance == 'aggressive' else 1.0
            score = sharpe * illiquidity_penalty * risk_adjustment
            scores.append((ticker, score, expected_return, mkt_beta, amihud, result))
        
        total_score = sum(score for _, score, _, _, _, _ in scores)
        weights = [score / total_score if total_score != 0 else 1.0 / len(tickers) for _, score, _, _, _, _ in scores]

        # Adjust weights based on risk tolerance
        if risk_tolerance == 'conservative':
            weights = [w / (beta + 1) for w, beta in zip(weights, [r[3] for r in scores])]  # Penalize high-beta stocks
        elif risk_tolerance == 'aggressive':
            weights = [w * expected_return for w, expected_return in zip(weights, [r[2] for r in scores])]  # Favor high-return stocks
        # For 'moderate', weights remain unchanged
        weights = np.array(weights)
        weights = weights / weights.sum()  # Normalize after adjustment
        
        weights = np.array(weights)
        weights = np.maximum(weights, min_weight)
        weights = np.minimum(weights, max_weight)
        weights /= weights.sum()
        
        portfolio_return = 0.0
        portfolio_betas = {factor: 0.0 for factor in significant_factors if factor != 'Mom'}
        portfolio_illiquidity = 0.0
        
        for (ticker, _, expected_return, mkt_beta, amihud, result), weight in zip(scores, weights):
            portfolio_return += weight * expected_return
            for factor in portfolio_betas:
                portfolio_betas[factor] += weight * result['Betas'].get(factor, 0.0)
            portfolio_illiquidity += weight * amihud
        
        sharpe_start_date = (pd.to_datetime(end_date) - pd.Timedelta(days=180)).strftime('%Y-%m-%d')
        portfolio_sharpes, sp500_sharpes, sharpe_labels = compute_historical_sharpe(tickers, weights, sharpe_start_date, end_date)
        
        # Generate dynamic insights
        insights = []
        top_stock = max(scores, key=lambda x: x[1])[0]
        top_result = next(result for result in stock_results if result['Ticker'] == top_stock)
        top_weight = [weight for (ticker, _, _, _, _, _), weight in zip(scores, weights) if ticker == top_stock][0]
        insights.append(
            f"Highest allocation to {top_stock} ({top_weight:.2%}) due to its strong risk-adjusted return "
            f"(Expected Return: {top_result['Expected Annual Return (%)']:.2f}%, Market Beta: {top_result['Betas']['Mkt-RF']:.2f})."
        )
        
        liquidity_info = []
        for ticker, _, _, _, amihud, result in scores:
            liquidity_desc = "highly illiquid - expect higher transaction costs" if amihud > 0.01 else \
                            "moderately liquid - manageable trading" if amihud > 0.0001 else \
                            "highly liquid - easy to trade"
            liquidity_info.append(f"{ticker}: {amihud:.6f} ({liquidity_desc})")
        insights.append("Liquidity Assessment (Amihud Illiquidity):\n  " + "\n  ".join(liquidity_info))
        
        risk_note = f"Portfolio balances risk and return for your moderate preference (Market Beta: {portfolio_betas['Mkt-RF']:.2f})."
        if risk_tolerance == 'aggressive':
            risk_note = f"This portfolio suits your aggressive stance, emphasizing higher returns with a market beta of {portfolio_betas['Mkt-RF']:.2f}."
            if market_regime == 'high':
                risk_note += " However, high market volatility (VIX ≥ 25) suggests caution - monitor for potential downturns."
        elif risk_tolerance == 'conservative':
            risk_note = f"Portfolio aligns with your conservative preference, with a market beta of {portfolio_betas['Mkt-RF']:.2f}."
            if market_regime == 'high':
                risk_note += " High market volatility (VIX ≥ 25) favors this defensive allocation."
        insights.append(risk_note)
        
        growth_value = "growth-oriented" if portfolio_betas['HML'] < 0 else "value-oriented"
        size_tilt = "large-cap" if portfolio_betas['SMB'] < 0 else "small-cap"
        insights.append(
            f"Portfolio Style: {growth_value} (HML: {portfolio_betas['HML']:.2f}) and {size_tilt}-focused "
            f"(SMB: {portfolio_betas['SMB']:.2f})."
        )
        
        insights.append(
            "Notes:\n- Premiums are based on 2017–2024 data and may reflect period-specific conditions.\n"
        )
        
        portfolio_summary = {
            'Allocations': {ticker: weight for (ticker, _, _, _, _, _), weight in zip(scores, weights)},
            'Portfolio Expected Return (%)': portfolio_return * 100,
            'Portfolio Factor Exposures': portfolio_betas,
            'Portfolio Amihud Illiquidity': portfolio_illiquidity,
            'Individual Stock Analysis': {result['Ticker']: result for result in stock_results},
            'Market Regime': market_regime,
            'Risk Tolerance': risk_tolerance,
            'Insights': insights,
            'Historical Sharpe': {'Portfolio': portfolio_sharpes, 'SP500': sp500_sharpes, 'Labels': sharpe_labels},
            'Weights': weights.tolist()
        }
        return portfolio_summary
    
    except Exception as e:
        print(f"Error suggesting portfolio: {str(e)}")
        return None


In [19]:
# --- Section 3: Run the App ---

In [44]:
# Step 1: Run data preparation (only needs to be run once)
# Paths to the data files
ff_path = os.path.join(notebook_dir, "F-F_Research_Data_5_Factors_2x3_daily.CSV")
mom_path = os.path.join(notebook_dir, "F-F_Momentum_Factor_daily.CSV")
illiq_path = os.path.join(notebook_dir, "illiq_factor_vw.csv")
crsp_path = os.path.join(notebook_dir, "daily stock price.csv")

# Run data preparation
all_premiums, significant_factors, ff_data = prepare_data(ff_path, mom_path, illiq_path, crsp_path)

Factor Correlations (2017–2024):
              Mkt-RF           SMB           HML       RMW           CMA  \
Mkt-RF  1.000000e+00  2.089147e-01 -1.412541e-01 -0.201180 -3.195120e-01   
SMB     2.089147e-01  1.000000e+00  2.763783e-01 -0.281037  3.804965e-02   
HML    -1.412541e-01  2.763783e-01  1.000000e+00  0.354558  5.926897e-01   
RMW    -2.011801e-01 -2.810375e-01  3.545584e-01  1.000000  2.927694e-01   
CMA    -3.195120e-01  3.804965e-02  5.926897e-01  0.292769  1.000000e+00   
Mom    -1.031017e-01 -1.395965e-16 -5.318439e-17 -0.045953  1.859249e-01   
ILLIQ   7.028495e-16  1.403455e-01 -3.909821e-16 -0.064968 -3.997497e-16   

                 Mom         ILLIQ  
Mkt-RF -1.031017e-01  7.028495e-16  
SMB    -1.395965e-16  1.403455e-01  
HML    -5.318439e-17 -3.909821e-16  
RMW    -4.595339e-02 -6.496769e-02  
CMA     1.859249e-01 -3.997497e-16  
Mom     1.000000e+00 -4.278330e-02  
ILLIQ  -4.278330e-02  1.000000e+00  

Fama-MacBeth Results (2017–2024):
Mkt-RF: Premium = 13.296009

In [39]:
# Step 2: Suggested Portfolio Feature
def main():
    """Entry point of the script, handling user input and displaying portfolio analysis results with interactive features.
    
    Parameters:
        None
    
    Returns:
        None
    
    Side Effects:
        Prints portfolio analysis, insights, and interactive prompts; saves charts to files.
    """
    tickers_input = input("Enter ticker symbols (e.g., META AAPL): ").strip().split()
    start_date = "2017-01-01"
    end_date = "2024-12-31"
    
    while True:
        risk_tolerance = input("Enter risk tolerance (conservative, moderate, aggressive): ").strip().lower()
        if risk_tolerance in ['conservative', 'moderate', 'aggressive']:
            break
        print("Invalid risk tolerance. Please enter 'conservative', 'moderate', or 'aggressive'.")
    
    portfolio_result = suggest_portfolio(tickers_input, start_date, end_date, risk_tolerance=risk_tolerance)
    
    if portfolio_result:
        current_vix = get_current_vix() 
        print("\n=== Portfolio Advisor Dashboard ===")
        if current_vix > 30:
            print("\nHigh VIX Detected: Consider rebalancing to preserve capital.")
        
        print(f"\nPortfolio Suggestion (2017–2024 premiums, Risk Tolerance: {portfolio_result['Risk Tolerance']}, Market Regime: {portfolio_result['Market Regime']}):")
        print("Allocations:")
        for ticker, weight in portfolio_result['Allocations'].items():
            print(f"  {ticker}: {weight:.2%}")
        print(f"Portfolio Expected Return: {portfolio_result['Portfolio Expected Return (%)']:.2f}%")
        print("Portfolio Factor Exposures (Weighted Betas):")
        for factor, beta in portfolio_result['Portfolio Factor Exposures'].items():
            print(f"  {factor}: {beta:.4f}")
        print(f"Portfolio Amihud Illiquidity: {portfolio_result['Portfolio Amihud Illiquidity']:.6f}")
        
        print("\nKey Insights:")
        for insight in portfolio_result['Insights']:
            print(f"- {insight}")
        
        print("\nSharpe Ratio Tracker:")
        plot_sharpe_tracker(
            portfolio_result['Historical Sharpe']['Portfolio'],
            portfolio_result['Historical Sharpe']['SP500'],
            portfolio_result['Historical Sharpe']['Labels']
        )
        
        # Automated Rebalancing
        print("\nAutomated Rebalancing:")
        print(f"Current VIX: {current_vix:.2f}")
        if current_vix > 30:
            rebalance_suggestion = "Switch to liquid stocks (Decile 0) to reduce drawdowns."
        elif current_vix < 18:
            rebalance_suggestion = "Switch to illiquid stocks (Decile 9) for higher returns."
        else:
            rebalance_suggestion = "Hold current portfolio."
        print(f"Suggestion: {rebalance_suggestion}")
        print("Options: [1] Activate Monthly Rebalance, [2] Execute Now, [3] Skip")
        choice = input("Enter your choice (1-3): ").strip()
        if choice == '1':
            print("Monthly rebalancing activated: Optimizing for Sharpe ratio with VIX strategy.")
        elif choice == '2':
            print(f"Rebalancing to: {rebalance_suggestion}")
        print("Note: Transaction costs considered in optimization.")
        
        # Risk Analyzer
        print("\nLiquidity Risk Assessment:")
        avg_amihud = portfolio_result['Portfolio Amihud Illiquidity']
        liquidity_risk = "High" if avg_amihud > 0.01 else "Moderate" if avg_amihud > 0.0001 else "Low"
        sharpe = (portfolio_result['Portfolio Expected Return (%)'] / 100 - 0.02) / portfolio_result['Portfolio Factor Exposures']['Mkt-RF']
        print(f"Liquidity Risk Exposure: {liquidity_risk} (Avg. Amihud: {avg_amihud:.6f})")
        print(f"Sharpe Ratio: {sharpe:.2f}")
        print("Options: [1] View Historical Performance, [2] Skip")
        choice = input("Enter your choice (1-2): ").strip()
        if choice == '1':
            print("Historical Performance:")
            print("Portfolio Sharpe Ratios:", [round(x, 2) for x in portfolio_result['Historical Sharpe']['Portfolio']])
            print("S&P 500 Sharpe Ratios:", [round(x, 2) for x in portfolio_result['Historical Sharpe']['SP500']])
            print("Months:", portfolio_result['Historical Sharpe']['Labels'])
        
        # VIX Insights
        print("\nVIX-Based Insights:")
        print(f"Current VIX: {current_vix:.2f} ({portfolio_result['Market Regime']} Volatility)")
        vix_suggestion = "Reduce exposure to illiquid stocks." if current_vix > 30 else \
                         "Increase exposure to illiquid stocks for higher returns." if current_vix < 18 else \
                         "Hold current portfolio."
        print(f"Suggestion: {vix_suggestion}")
        print("Options: [1] Rebalance Now, [2] Skip")
        choice = input("Enter your choice (1-2): ").strip()
        if choice == '1':
            print(f"Rebalancing: {vix_suggestion}")
        
        # Individual Stock Analysis
        print("\nIndividual Stock Analysis:")
        for ticker, result in portfolio_result['Individual Stock Analysis'].items():
            print(f"\nPortfolio Advice for {result['Ticker']} using Multi-Factor Model (2017–2024 premiums):")
            print(f"Period: {result['Period']}")
            print(f"Amihud Illiquidity: {result['Amihud Illiquidity']:.6f}")
            print("Factor Betas and Significance:")
            for factor in result['Betas']:
                print(f"  {factor}: Beta = {result['Betas'][factor]:.4f}, p-value = {result['Beta P-Values'][factor]:.4f}")
            print("Factor Contributions (%):")
            for factor, contrib in result['Factor Contributions (%)'].items():
                print(f"  {factor}: {contrib:.4f}")
            print(f"Expected Annual Return: {result['Expected Annual Return (%)']:.2f}%")

if __name__ == "__main__":
    main()

Enter ticker symbols (e.g., META AAPL):  META AAPL
Enter risk tolerance (conservative, moderate, aggressive):  aggressive


Current VIX: 22.07, Market Regime: medium
Note: const beta for META is not significant (p = 0.6305).
Note: Mom beta for META is not significant (p = 0.1826).
Note: ILLIQ beta for META is not significant (p = 0.7818).
Note: Capped Mom premium at 10% for META (original: 52.45192976%).
Note: const beta for AAPL is not significant (p = 0.2596).
Note: Mom beta for AAPL is not significant (p = 0.0617).
Note: ILLIQ beta for AAPL is not significant (p = 0.5481).
Note: Capped Mom premium at 10% for AAPL (original: 52.45192976%).

=== Portfolio Advisor Dashboard ===

Portfolio Suggestion (2017–2024 premiums, Risk Tolerance: aggressive, Market Regime: medium):
Allocations:
  META: 43.76%
  AAPL: 56.24%
Portfolio Expected Return: 21.26%
Portfolio Factor Exposures (Weighted Betas):
  Mkt-RF: 1.3493
  SMB: -0.2298
  HML: -0.4281
  RMW: 0.3745
  CMA: -0.2086
  ILLIQ: -0.0033
Portfolio Amihud Illiquidity: 0.000002

Key Insights:
- Highest allocation to AAPL (56.24%) due to its strong risk-adjusted ret

Enter your choice (1-3):  3


Note: Transaction costs considered in optimization.

Liquidity Risk Assessment:
Liquidity Risk Exposure: Low (Avg. Amihud: 0.000002)
Sharpe Ratio: 0.14
Options: [1] View Historical Performance, [2] Skip


Enter your choice (1-2):  3



VIX-Based Insights:
Current VIX: 22.07 (medium Volatility)
Suggestion: Hold current portfolio.
Options: [1] Rebalance Now, [2] Skip


Enter your choice (1-2):  3



Individual Stock Analysis:

Portfolio Advice for META using Multi-Factor Model (2017–2024 premiums):
Period: 2017-01-01 to 2024-12-31
Amihud Illiquidity: 0.000003
Factor Betas and Significance:
  const: Beta = 0.0002, p-value = 0.6305
  Mkt-RF: Beta = 1.3263, p-value = 0.0000
  SMB: Beta = -0.2585, p-value = 0.0087
  HML: Beta = -0.3319, p-value = 0.0004
  RMW: Beta = 0.2011, p-value = 0.0486
  CMA: Beta = -0.9129, p-value = 0.0000
  Mom: Beta = -0.0808, p-value = 0.1826
  ILLIQ: Beta = 0.0138, p-value = 0.7818
Factor Contributions (%):
  Mkt-RF: 17.6345
  SMB: -0.3860
  HML: 1.0846
  RMW: -0.1850
  CMA: 2.2629
  Mom: -0.8075
  ILLIQ: 0.0170
Expected Annual Return: 21.71%

Portfolio Advice for AAPL using Multi-Factor Model (2017–2024 premiums):
Period: 2017-01-01 to 2024-12-31
Amihud Illiquidity: 0.000001
Factor Betas and Significance:
  const: Beta = 0.0003, p-value = 0.2596
  Mkt-RF: Beta = 1.3671, p-value = 0.0000
  SMB: Beta = -0.2075, p-value = 0.0008
  HML: Beta = -0.5030, p-val